# HowTo Load a .npy File

In [1]:
import numpy as np
import tensorflow as tf
import struct

/home/nokia-du/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Create a random numpy array and save as .npy file

In [2]:
save_file = '../datasets/numpy_array/numpy_random_array'

In [38]:
save_file = '../datasets/numpy_array/numpy_random_array'
numpy_random_array = np.random.rand(5, 9)
print(numpy_random_array.dtype)
print('Shape:', numpy_random_array.shape)
print('[0]', numpy_random_array[0])
print('[-1]', numpy_random_array[-1])
np.save(save_file, numpy_random_array)
print(save_file + '.npy saved')

float64
Shape: (5, 9)
[0] [ 0.11272442  0.77521092  0.4789582   0.54178528  0.94407005  0.6263302
  0.75687382  0.64781394  0.00172408]
[-1] [ 0.62241592  0.08582086  0.07876771  0.9516751   0.60793591  0.78781025
  0.17234172  0.72442766  0.73736114]
../datasets/numpy_array/numpy_random_array.npy saved


## Use np.load() to load file

In [39]:
restore_random_array = np.load(save_file + '.npy')
print(restore_random_array.dtype)
print('Shape:', restore_random_array.shape)
print('[0]:', restore_random_array[0])
print('[-1]:', restore_random_array[-1])

float64
Shape: (5, 9)
[0]: [ 0.11272442  0.77521092  0.4789582   0.54178528  0.94407005  0.6263302
  0.75687382  0.64781394  0.00172408]
[-1]: [ 0.62241592  0.08582086  0.07876771  0.9516751   0.60793591  0.78781025
  0.17234172  0.72442766  0.73736114]


## Now we are going to hack the npy

### First, understand the npy format

[A Simple File Format for NumPy Arrays](https://docs.scipy.org/doc/numpy-dev/neps/npy-format.html)

According the document, we know that the real value is start with index **80**.
So we can use **tf.read_file()** to read file and get all strings, then we can use **tf.substr()** to get the real value.

In [51]:
# You have to know the length or read the length from .npy file
tf_file_string_shape = restore_random_array.shape[0] * restore_random_array.shape[1]
# You have to know the format from .npy file
# If the npy file format is float, you should set it for 4, and file format for '<f'
tf_file_format_size = 8

tf_file_name = tf.constant(save_file + '.npy')
tf_file_string = tf.read_file(tf_file_name)
# with tf.Session() as sess:
#     print(sess.run(tf_file_string))
    
tf_file_string = tf.substr(tf_file_string, [80], [tf_file_string_shape * 8])[0]
with tf.Session() as sess:
    print(sess.run(tf_file_string))

b'\x08\x8e\xe7\xdb\x81\xdb\xbc?\xd2c\xbf"\x87\xce\xe8?\x9a\x84\xb4F@\xa7\xde?r\xf7h\x14NV\xe1?gO\xc5e\xd25\xee?`q\\\xa3\xe5\n\xe4?\xac1}rO8\xe8?\tv\xa7K\xe4\xba\xe4?\x00x]\xb7Q?\\?\xde\xd4r\xed\xdf\x97\xd4??W!@}$\xed?L`\xdf{dH\xe6?\xac\x0b\x95\x85\x8f\xc0\xc2?\x0e\xd3\xeac\x87T\xe9?\x08o\x91_i\xc9\xd9?3{\xf7\xf8\x17g\xe3?\x99k\xdb\xb7\xdf\xf2\xe7?\x02p\xa0i\x13\xd9\xd1?\x8c\xab$\xde\xe2\x81\xc7?hDp\x90_5\xd3?q\xa4\xc0\xb2/p\xe2?L\xb0l\xa6\xab\x1d\xe6?L\nA[\xf9H\xdd?Mfk1\xfcW\xe1?\xf0\xf2\x19\xe3F\xb1\xa3?\xa0\xf6>\xaeb9\x96?8x\x03Q:\t\xda?\xe0F\xf0\x06\xc6\xb3\xd7?"\x95f\xa13[\xe8?\xd0\xeb\x99\x05\x93\xe3\xed?\x8a\xc1\xdd<\xbf\\\xe1?tt\xe4\x9f\x9e\x05\xdc?X\xeb\x8d\x9c\x10\x9d\xb6?a;\xd7\r\xear\xed?\x84\x7f\x80\xef\xc7\xfb\xcc?t\xb7].5\x9e\xc9?\xa2\xf8O\xc9\xd4\xea\xe3?X\xbaM$[\xf8\xb5?\x88\xd8)\xe0\x1e*\xb4?\xc0[\nU\x1ft\xee?\xbc\x0f\x94\x046t\xe3?\xee[\x04\xd5\xbd5\xe9?\\\xd8\x8f#K\x0f\xc6?\xd2\x86\xa5\xec\x82.\xe7?u~\xa3ev\x98\xe7?'


### Second, Convert byte to float

We need call python function to help converting. We can use **tf.py_func()** and **struct.unpack()** to do this.

In [52]:
# You have to know the format from .npy file
tf_file_format = tf.constant('<' + 'd' * tf_file_string_shape)

# Be careful, you have to use tf.float64 for converting
tf_random_array = tf.py_func(struct.unpack, [tf_file_format, tf_file_string], [tf.float64] * tf_file_string_shape)
tf_random_array = tf.stack(tf_random_array, axis=0)
tf_random_array = tf.cast(tf_random_array, tf.float32)
with tf.Session() as sess:
    tf_random_array_final = sess.run(tf_random_array)

tf_random_array_final = tf_random_array_final.reshape((5, 9))
print('shape:', tf_random_array_final.shape)
print('dtype:', tf_random_array_final.dtype)
print('[0]:', tf_random_array_final[0])
print('[-1]:', tf_random_array_final[-1])

shape: (5, 9)
dtype: float32
[0]: [ 0.11272442  0.77521092  0.47895819  0.5417853   0.94407004  0.6263302
  0.75687385  0.64781392  0.00172408]
[-1]: [ 0.6224159   0.08582086  0.07876771  0.95167512  0.60793591  0.78781027
  0.17234172  0.72442764  0.73736113]
